In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

#from mtflearn import ZPs

In [2]:
from mtflearn.features import autocorrelation, radial_profile, get_characteristic_length
from mtflearn.utils import normalize_image

In [2]:
from skimage.transform import resize

In [70]:
filename = 'data\\monolayer_MoSe2_80K.npy'
img = np.load(filename)[0:800, 0:800]
img = resize(img, (512, 512))
img = normalize_image(img, 0, 1)

In [71]:
get_characteristic_length(img)

27

In [74]:
img = np.load(filename)

In [81]:
from skimage.transform import warp_polar

def baseline_correction(y,niter=10):
    n = len(y)
    y_ = np.log(np.log(np.sqrt(y +1)+1)+1)
    yy = np.zeros_like(y)

    for pp in np.arange(1,niter+1):
        r1 = y_[pp:n-pp]
        r2 = (np.roll(y_,-pp)[pp:n-pp] + np.roll(y_,pp)[pp:n-pp])/2
        yy = np.minimum(r1,r2)
        y_[pp:n-pp] = yy

    baseline = (np.exp(np.exp(y_)-1)-1)**2 -1
    return baseline

def get_fft_line(img):
    fft_abs = np.abs(np.fft.fftshift(np.fft.fft2(img)))
    fft_log = np.log(fft_abs + 1)
    i ,j = np.unravel_index(np.argmax(fft_log), shape=img.shape)
    print(i, j)
    line = warp_polar(fft_log, center=(i, j)).mean(axis=0)[0:i]
    # get the baseline
    baseline = baseline_correction(line)
    # remove the baseline
    line_ = line - baseline
    return line_

In [80]:
ll = get_fft_line(img)
plt.plot(ll)

1024 1024


In [77]:
2048/56.7 *0.64

23.11675485008818

In [58]:
27*np.sqrt(3) 

46.76537180435968

In [72]:
hh = autocorrelation(img)
ll = radial_profile(hh)

In [73]:
plt.plot(ll)
plt.axhline(ll.mean())

In [17]:
from scipy.signal import find_peaks

In [18]:
find_peaks(ll)

In [22]:
peaks, _ = find_peaks(ll, height=0)
plt.plot(peaks, ll[peaks], "x")
plt.plot(ll)
plt.axhline(ll.mean())